In [ ]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
from crawlers.models import *
from crawlers.finlab.pioneers import *
from crawlers.finlab.import_tools import *
import time

In [ ]:
class CrawlBrokerTradeTW:
    
    def __init__(self,start_date):
        self.start_date = start_date
        self.start_date_str = start_date.strftime("%Y-%m-%d")
        self.target_name = "台股分點進出資訊"
        self.format = "time_series"
        
    
    def check_trade_day(self):
        stock_range=CrawlStockPriceTW(self.start_date)
        try:
            df=pd.concat([stock_range.crawl_sii(),stock_range.crawl_otc()])['stock_id'].values
            return df
        except ValueError:
            return None
        
    def broker_trade(self,stock_id):
        print(stock_id)
        url='https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco.djhtm?a='+stock_id+'&e='+self.start_date_str+'&f='+self.start_date_str
        r = requests.post(url)
        html_df = pd.read_html(StringIO(r.text))
        df = pd.DataFrame(html_df[2])
        #holiday
        if len(df)<9:
            return  None
        df.columns=df.iloc[5]
        df=df.iloc[6:-3]
        buy_side=df.iloc[:,:5]
        buy_side=buy_side.rename(columns={'買超券商':'broker_name','買進':'buy_num',
                                          '賣出':'sell_num','買超':'net_bs',
                                          '佔成交比重':'transactions_pt'})
        sell_side=df.iloc[:,5:]
        sell_side=sell_side.rename(columns={'賣超券商':'broker_name','買進':'buy_num',
                                            '賣出':'sell_num','賣超':'net_bs',
                                            '佔成交比重':'transactions_pt'})

        df_all=pd.concat([buy_side,sell_side],sort=False).dropna()
        df_all.iloc[:,1:]=df_all.iloc[:,1:].apply(lambda s: pd.to_numeric(s.str.replace('%',''), errors="coerce"))
        df_all['net_bs']=df_all['buy_num']-df_all['sell_num']
        df_all["date"] = pd.to_datetime(self.start_date)
        df_all['broker_name']=df_all['broker_name'].apply(lambda s:s.replace('證券','')).apply(lambda s:s.replace('(牛牛牛)','犇'))
        df_all['broker_name']=df_all['broker_name'].apply(lambda s:s if '停' not in s else s[:s.index('-')])
        df_all['stock_id']=df_all['broker_name'].apply(lambda s:stock_id+'-'+s)
#         add_to_sql(BrokerTradeTW, df_all, 'broker_name')
        
        return df_all

    def crawl_main(self):
        crawl_list=self.check_trade_day()
        if crawl_list is not None:
            for stock_id in crawl_list[:5]:
                 self.broker_trade(stock_id)
        else:
            pass



# a=CrawlBrokerTradeTW(datetime.datetime(2020,4,23))
# a.broker_trade('00774B')
# df=broker_trade(stock_id,datetime.datetime(2020,4,23),end_time=None)
# df

a=CrawlBrokerTradeTW(datetime.datetime(2018,1,2))
# a.broker_trade('1101')['date']
a.crawl_main()

In [ ]:
df=StockPriceTW.objects.filter(date=datetime.datetime(2020,4,24))

In [ ]:
# [i['stock_id'] for i in df.values('stock_id')]
stock_range=CrawlStockPriceTW(date=datetime.datetime(2020,4,25))
try:
    df=pd.concat([stock_range.crawl_sii(),stock_range.crawl_otc()])['stock_id'].values
except ValueError:
    return None

In [ ]:
add_to_sql(BrokerTradeTW, df, 'broker_name')

In [ ]:
BrokerInfoTW.objects.filter(broker_name='臺銀')

# 匯入foreign,測試

In [ ]:
model_name=BrokerTradeTW

fk_field_names = [field.name for field in model_name._meta.fields
                             if isinstance(field, models.ForeignKey)]

fk_field_names

In [ ]:
fk_remote=[model_name._meta.get_field(i).remote_field.model  for i in fk_field_names]
fk_remote

In [ ]:
model_name=BrokerTradeTW
fk_columns='broker_name'

def fk_import(model_name,item,*fk_columns):
    fk_field_names = [field.name for field in model_name._meta.fields
                      if isinstance(field, models.ForeignKey)]
    fk_remote = [model_name._meta.get_field(i).remote_field.model for i in fk_field_names]
    fk_obj = [m.objects.get(**{n: item[n]}) for m, n in zip(fk_remote, fk_columns)]
    fk_create_data = dict((m, n) for m, n in zip(fk_field_names, fk_obj))
    return  fk_create_data


fk_import(model_name,df.iloc[0],fk_columns)

In [ ]:
model_name=BrokerTradeTW
fk_columns='broker_name'

def fk_update(model_name,item,*fk_columns):
    fk_field_names = [field.name for field in model_name._meta.fields
                      if isinstance(field, models.ForeignKey)]
    fk_remote = [model_name._meta.get_field(i).remote_field.model for i in fk_field_names]
    fk_obj = [m.objects.get(**{n: item[n]}) for m, n in zip(fk_remote, fk_columns)]

    return fk_obj 


fk_update(model_name,df.iloc[0],fk_columns)